In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import librosa
import os
from sklearn.model_selection import train_test_split

# Dataset tùy chỉnh cho dữ liệu âm thanh
class AudioDataset(Dataset):
    def __init__(self, file_paths, labels, transform=None):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        # Load âm thanh và giới hạn về 10 giây
        audio, sr = librosa.load(file_path, sr=None, duration=5)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfcc = np.expand_dims(mfcc, axis=0)  # Thêm chiều cho kênh
        if self.transform:
            mfcc = self.transform(mfcc)
        return torch.tensor(mfcc, dtype=torch.float32), torch.tensor(label, dtype=torch.long)



    def __getitem2__(self, idx):
        file_path = self.file_paths[idx]
        label = self.labels[idx]
        # Load âm thanh và trích xuất đặc trưng MFCC
        audio, sr = librosa.load(file_path, sr=None)
        mfcc = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=40)
        mfcc = np.expand_dims(mfcc, axis=0)  # Thêm chiều cho kênh
        if self.transform:
            mfcc = self.transform(mfcc)
        return torch.tensor(mfcc, dtype=torch.float32), torch.tensor(label, dtype=torch.long)

# Thiết kế mô hình CNN bằng PyTorch với Dropout để giảm overfitting
class AudioCNN(nn.Module):
    def __init__(self):
        super(AudioCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(0.5)  # Thêm Dropout với tỷ lệ 0.5
        self.fc1 = None  # Sẽ được khởi tạo sau
        self.fc2 = nn.Linear(128, 4)  # 4 lớp âm thanh

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        if self.fc1 is None:
            self.fc1 = nn.Linear(x.size(1), 128).to(x.device)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Áp dụng dropout trước lớp cuối cùng
        x = self.fc2(x)
        return x

# Tạo mô hình
model = AudioCNN()

# Optimizer và hàm loss
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()

# Lấy danh sách file âm thanh và kiểm tra nhãn
audio_dir = r"E:\dataset\sound-data\sound-data\audio"
file_paths = [os.path.join(audio_dir, file) for file in os.listdir(audio_dir) if file.endswith(".wav")]
labels = [0 for _ in range(len(file_paths))]  # Cập nhật nhãn phù hợp nếu có

# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
train_paths, test_paths, train_labels, test_labels = train_test_split(
    file_paths, labels, test_size=0.3, stratify=labels, random_state=42)

# Tạo dataset và dataloader cho tập huấn luyện và kiểm tra
train_dataset = AudioDataset(train_paths, train_labels)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = AudioDataset(test_paths, test_labels)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Tải mô hình nếu đã có mô hình được lưu trước đó
if os.path.exists("best_audio_classification_model.pth"):
    model.load_state_dict(torch.load("best_audio_classification_model.pth"), strict=False)
    print("Đã tải mô hình từ file 'best_audio_classification_model.pth'")

# Huấn luyện mô hình
model.train()
epoch = 0
best_accuracy = 0.0


# Đặt số lượng epoch cố định
num_epochs = 50

# Huấn luyện mô hình
model.train()
best_accuracy = 0.0
for epoch in range(num_epochs):
    running_loss = 0.0
    total_batches = len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs} - Tổng số batch trong tập huấn luyện: {total_batches}")

    for i, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # In đường dẫn của các file âm thanh trong batch hiện tại
        batch_file_paths = [train_paths[idx] for idx in range(i * train_loader.batch_size, min((i+1) * train_loader.batch_size, len(train_paths)))]
        print(f"Batch {i+1}/{total_batches} - Đang huấn luyện trên các file âm thanh: {batch_file_paths}")

        # In ra sau mỗi 10 batch
        if (i + 1) % 10 == 0:
            print(f"Batch {i+1}/{total_batches} - Loss trung bình: {running_loss / (i + 1)}")

    print(f"Epoch {epoch+1}, Loss trung bình: {running_loss / total_batches}")
    
    # Kiểm tra độ chính xác trên tập kiểm tra
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for j, (inputs, targets) in enumerate(test_loader):
            outputs = model(inputs)
            predicted = torch.argmax(outputs, dim=1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
            # In đường dẫn của các file âm thanh trong batch kiểm tra hiện tại
            batch_file_paths = [test_paths[idx] for idx in range(j * test_loader.batch_size, min((j+1) * test_loader.batch_size, len(test_paths)))]
            print(f"Batch kiểm tra {j+1}/{len(test_loader)} - Đang kiểm tra trên các file âm thanh: {batch_file_paths}")
            
    accuracy = 100 * correct / total
    print(f"Độ chính xác trên tập kiểm tra: {accuracy}%")
    model.train()
    
    # Lưu mô hình nếu đạt độ chính xác tốt hơn
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        torch.save(model.state_dict(), "best_audio_classification_model.pth")
        print("Đã lưu mô hình tốt nhất vào file 'best_audio_classification_model.pth'")
    
    # Nếu đạt độ chính xác tối đa thì kết thúc sớm
    if accuracy >= 100.0:
        print("Đạt độ chính xác tối đa. Kết thúc sớm quá trình huấn luyện.")
        break

print("Huấn luyện hoàn tất!")



# Đánh giá mô hình
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        predicted = torch.argmax(outputs, dim=1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f"Độ chính xác của mô hình: {100 * correct / total}%")
